# Erstellen und Trainieren der KI

In [1]:
# Needed imports
import numpy as np
from pathlib import Path
from tensorflow import keras

# Needed variabel
dataset_folder = Path(__file__).parent / "dataset"
sequence_length = 64
model_path = Path(__file__).parent / "model.h5"

2023-02-06 08:26:42.637305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 08:26:42.637326: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Load the dataset
songs = np.load(dataset_folder / "songs.npy")

print(songs.shape)
print(songs[:10])

songs = songs.tolist()

(2576,)
[13 16 16 16 16 16 12 16 11 16]


## Erstellen der Trainingsdaten
Wir erstellen sequenzen von eienr bestimmten länge und sagen dem Netzwerk, ok was kommt als nächstes. Dies macht man so lange, bis man alle Daten durchhat. Das schafft man, indem man die die Trainignsdaten nimmt, ein Teil herausschneided und dan den Ausschnit zum Herusschneiden immer eins weiter nach rechts shifted, bis man alle Daten durch ist.

```bash
# Data
[12, 13, 14, 15, 16, 17, ...]

# Erster input und targed
[12, 13] -> 14

#Zweiter input und targed
[13, 14] -> 15
```

In [3]:
# 100 symbols, 64 sl
# 100 - 64 = 36
num_sequences = len(songs) - sequence_length

inputs = []
targets = []

for i in range(num_sequences):
    inputs.append(songs[i:i+sequence_length])
    targets.append(songs[i+sequence_length])

In [4]:
print(f"Num sequences: {len(inputs)}")
print("the first input sequence:")
print(inputs[0])
print("the first target sequence:")
print(targets[0])
print()
print("The whole dataset for the first input sequence:")
print(songs[:sequence_length+1])
print()
print(len(inputs), len(targets))

Num sequences: 2512
the first input sequence:
[13, 16, 16, 16, 16, 16, 12, 16, 11, 16, 5, 16, 9, 16, 16, 16, 7, 16, 6, 16, 4, 16, 16, 16, 9, 16, 16, 16, 7, 16, 6, 16, 4, 16, 16, 16, 9, 16, 16, 16, 7, 16, 5, 16, 3, 16, 16, 17, 13, 16, 16, 16, 16, 16, 12, 16, 11, 16, 5, 16, 9, 16, 16, 16]
the first target sequence:
7

The whole dataset for the first input sequence:
[13, 16, 16, 16, 16, 16, 12, 16, 11, 16, 5, 16, 9, 16, 16, 16, 7, 16, 6, 16, 4, 16, 16, 16, 9, 16, 16, 16, 7, 16, 6, 16, 4, 16, 16, 16, 9, 16, 16, 16, 7, 16, 5, 16, 3, 16, 16, 17, 13, 16, 16, 16, 16, 16, 12, 16, 11, 16, 5, 16, 9, 16, 16, 16, 7]

2512 2512


## One-Hot encoding
Jeder mögliche Wert den es geben kann wird als Reihe in einer tabelle gesehen.
![](./Bilder/oenHot.png)

In unserem Fall ist ist die größe der Tabelle gleich der Anzahl an Elemente in unserem `mapping.json`. In unserem Fall sollte das 18 sein.

In [5]:
print(len(inputs))

2512


In [6]:
vacabulay_size = len(np.unique(songs))
inputs = keras.utils.to_categorical(inputs, num_classes=vacabulay_size)
targets = np.array(targets)

In [7]:
print(f"Vocabulary size: {vacabulay_size}")
print(inputs.shape)
print()
print(inputs)

Vocabulary size: 18
(2512, 64, 18)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 ...

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  

In [8]:
print(targets.shape)
print()
print(targets)

(2512,)

[ 7 16  6 ...  0  0  0]


## Erstellen des Modells


In [9]:
output_layer_size = vacabulay_size  # Number of unique symbols in the dataset
hidden_layer_sizes = [256]      # Number of neurons in each hidden layer
loss = "sparse_categorical_crossentropy"    # Loss function
activation = "softmax"                      # Activation function
learning_rate = 0.001                       # Learning rate
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)  # Optimizer

Hier erstellt man nun das konkrete Netzwerk.

`shape = (None, output_layer_size)`
1. Variable `None`-> Wie viele Zeitstemnpel hat man. Bei None wird das automatisch ermittelt 
2. Variable `output_layer_size` -> Wie viele Inputs hat man pro Eingabe. Dies ist hier die Anzahl der verwendeten Noten

Dem layer wird dann hidden LSTM layer hinzugefügt und ein Dropout, welches gegen Overfitting hilft.

In [10]:
# create the model achitecture
# model = keras.Sequential() # <- Would be a good idea to use this

# Input layer
input_layer = keras.layers.Input(shape=(None, output_layer_size))

# Hidden layers
x = input_layer
for hidden_layer_size in hidden_layer_sizes:
    x = keras.layers.LSTM(hidden_layer_size)(x)

# Avoid overfitting
x = keras.layers.Dropout(0.3)(x)

# Output layer. Full connection -> Dense
output_layer = keras.layers.Dense(output_layer_size, activation=activation)(x)

# The actual model
model = keras.Model(input_layer, output_layer)

2023-02-06 08:26:43.956150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-06 08:26:43.956178: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-06 08:26:43.956198: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5193325dcd0b): /proc/driver/nvidia/version does not exist
2023-02-06 08:26:43.956459: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# Compile the model
model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 18)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               281600    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 18)                4626      
Total params: 286,226
Trainable params: 286,226
Non-trainable params: 0
_________________________________________________________________


In [13]:
# keras.utils.plot_model(model, 'model.png', show_shapes=True)

## Trainiere das Modell

In [14]:
print(inputs.shape)
print(targets.shape)

shape = inputs[0].shape
for i, x in enumerate(inputs):
    if x.shape != shape:
        print(f"Error: {i}")

(2512, 64, 18)
(2512,)


In [15]:
epochs = 50
batch_size = 64  # Equal to sequence_length

result = model.fit(inputs, targets, epochs=epochs, batch_size=batch_size)

Epoch 1/50


2023-02-06 08:26:44.274905: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


79/79 [==============================] - 5s 42ms/step - loss: 1.3686 - accuracy: 0.7134
Epoch 2/50
79/79 [==============================] - 3s 41ms/step - loss: 1.1215 - accuracy: 0.7512
Epoch 3/50
79/79 [==============================] - 3s 39ms/step - loss: 1.0539 - accuracy: 0.7600
Epoch 4/50
79/79 [==============================] - 3s 39ms/step - loss: 1.0269 - accuracy: 0.7627
Epoch 5/50
79/79 [==============================] - 3s 41ms/step - loss: 0.9697 - accuracy: 0.7643
Epoch 6/50
79/79 [==============================] - 3s 42ms/step - loss: 0.9522 - accuracy: 0.7667
Epoch 7/50
79/79 [==============================] - 3s 40ms/step - loss: 0.8488 - accuracy: 0.7643
Epoch 8/50
79/79 [==============================] - 3s 40ms/step - loss: 0.7920 - accuracy: 0.7667
Epoch 9/50
79/79 [==============================] - 3s 40ms/step - loss: 0.7660 - accuracy: 0.7707
Epoch 10/50
79/79 [==============================] - 3s 40ms/step - loss: 0.7129 - accuracy: 0.7803
Epoch 11/50
79/79 [=

## Speicher das Modell

In [17]:
model.save(model_path)